In [64]:
#import libraries
from langchain.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os


In [74]:
#load credentials
load_dotenv(override=True)


True

In [32]:
#define model
LLM=ChatOpenAI(model="gpt-4o-mini")

In [33]:
#embedding_model
embedding_model=OpenAIEmbeddings(model="text-embedding-3-small")


C:\Users\redhu\AppData\Local\Temp\ipykernel_26700\4232507645.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model=OpenAIEmbeddings(model="text-embedding-3-small")


## 1. Data Injection

In [4]:
#get the data data path
os.getcwd() 

'c:\\Users\\redhu\\OneDrive\\Desktop\\document_portal\\notebook'

In [13]:
file_path=os.path.join(os.getcwd(), "data", "sample.pdf")

In [14]:
loader=PyPDFLoader(file_path)


In [15]:
documents=loader.load()

In [16]:
len(documents)

7

In [17]:
documents

[Document(metadata={'producer': 'Skia/PDF m136', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36 Edg/136.0.0.0', 'creationdate': '2025-05-10T23:15:04+00:00', 'title': "Understanding LSTM Networks -- colah's blog", 'moddate': '2025-05-10T23:15:04+00:00', 'source': 'c:\\Users\\redhu\\OneDrive\\Desktop\\document_portal\\notebook\\data\\sample.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='Understanding LSTM Networks\nPosted on August 27, 2015\nRecurrent Neural Networks\nHumans don’t start their thinking from scratch every second. As you read this essay, you understand each word basedon your understanding of previous words. You don’t throw everything away and start thinking from scratch again. Yourthoughts have persistence.\nTraditional neural networks can’t do this, and it seems like a major shortcoming. For example, imagine you want toclassify what kind of event is happening at every point i

## Text splitting

In [19]:
#texr splitting
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50,length_function=len)

In [20]:
docs=text_splitter.split_documents(documents)

In [22]:
len(docs)

45

In [23]:
docs[0].page_content

'Understanding LSTM Networks\nPosted on August 27, 2015\nRecurrent Neural Networks\nHumans don’t start their thinking from scratch every second. As you read this essay, you understand each word basedon your understanding of previous words. You don’t throw everything away and start thinking from scratch again. Yourthoughts have persistence.'

In [25]:
docs[0].metadata

{'producer': 'Skia/PDF m136',
 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36 Edg/136.0.0.0',
 'creationdate': '2025-05-10T23:15:04+00:00',
 'title': "Understanding LSTM Networks -- colah's blog",
 'moddate': '2025-05-10T23:15:04+00:00',
 'source': 'c:\\Users\\redhu\\OneDrive\\Desktop\\document_portal\\notebook\\data\\sample.pdf',
 'total_pages': 7,
 'page': 0,
 'page_label': '1'}

## Embedding and store the Embedded Data

In [34]:
vector_store=FAISS.from_documents(docs, embedding_model)

In [35]:
len(embedding_model.embed_documents(docs[0].page_content)[0])

1536

In [41]:
relevent_documents=vector_store.similarity_search("What is the main idea of the document?",k=2)

In [42]:
relevent_documents[1].page_content

'In the above diagram, each line carries an entire vector, from the output of one node to the inputs of others. The pinkcircles represent pointwise operations, like vector addition, while the yellow boxes are learned neural network layers.Lines merging denote concatenation, while a line forking denote its content being copied and the copies going todifferent locations.\nThe Core Idea Behind LSTMs\nThe key to LSTMs is the cell state, the horizontal line running through the top of the diagram.'

In [ ]:
#store the embbededd data on disk
vector_store.save_local("faiss_index")

In [ ]:
#set vector as retriever
retriever=vector_store.as_retriever()



In [46]:
retriever.invoke("What is the main idea of the document?")

[Document(id='2a8c6410-ff67-4533-bc02-911363e8ef08', metadata={'producer': 'Skia/PDF m136', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36 Edg/136.0.0.0', 'creationdate': '2025-05-10T23:15:04+00:00', 'title': "Understanding LSTM Networks -- colah's blog", 'moddate': '2025-05-10T23:15:04+00:00', 'source': 'c:\\Users\\redhu\\OneDrive\\Desktop\\document_portal\\notebook\\data\\sample.pdf', 'total_pages': 7, 'page': 1, 'page_label': '2'}, page_content='who found some pretty fundamental reasons why it mightbe difficult.'),
 Document(id='9d679b73-8073-4023-8901-8cdd5364cefb', metadata={'producer': 'Skia/PDF m136', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36 Edg/136.0.0.0', 'creationdate': '2025-05-10T23:15:04+00:00', 'title': "Understanding LSTM Networks -- colah's blog", 'moddate': '2025-05-10T23:15:04+00:00', 'source': 'c:\\Users\\redh

In [50]:
prompt_template="""Answer the question based on the context provided.
if you don't have sufficient information, just say "I don't have the information"

context: {context}

Question: {question}

Answer:
"""

In [ ]:

prompt=PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [52]:
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based on the context provided.\nif you don\'t have sufficient information, just say "I don\'t have the information"\n\ncontext: {context}\n\nQuestion: {question}\n\nAnswer:\n')

In [57]:
def format_docs(docs):
    return "/n/n".join([doc.page_content for doc in docs])

In [69]:
reg_chain=(
        {"context":retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | LLM
        | StrOutputParser()
    )

In [75]:
reg_chain.invoke("What is the Variants on Long Short Term Memory?")

'Variants on Long Short Term Memory (LSTM) are slight modifications and different versions of the standard LSTM architecture that have been proposed in various research papers. These variants often address specific tasks or improve certain aspects of the basic LSTM model. Notable examples include Depth Gated RNNs and Clockwork RNNs. Research has shown that while there are multiple variants, their performance is generally similar, with some architectures potentially outperforming LSTMs for specific tasks.'